# Parte 1 - Coleta de dados

A FIDE não disponibiliza uma base de dados unificada com o rating dos jogadores ao longo dos anos. Por isso, essa primeira parte do notebook é para montar essa base.

Ao final desse processo, será gerado um arquivo `ratings.json`, contendo o top-100 de jogadores ao longo dos anos, com a seguinte estrutura:

```python
[
  {
    'date_human': 'January 2018',
    'year': 2018,
    'month': 'January',
    'ratings': [
      ['Carlsen, Magnus', 2834],
      ['Caruana, Fabiano', 2811],
      # ...
    ]
  },
  # ...
]
```

Essa etapa **não** precisa ser exucutada. O arquivo `ratings.json` já está disponível no repositório.

Esse código só está aqui para fins de histórico e pro caso de se querer estender a base de dados para anos anteriores a **2013** (quando a FIDE começou a disponibilizar o ranking 1x por mês).

In [1]:
# import json
# import requests
# from bs4 import BeautifulSoup

In [2]:
# URL_TEMPLATE = "https://ratings.fide.com/toparc.phtml?cod={{ cod }}"

# COD_START = 241 # jan-2013
# COD_END = 761 # nov-2023

# all_cods = []

# cod = COD_START
# while cod <= COD_END:
#     all_cods.append(cod)
#     cod += 4

# num_cods = len(all_cods)

# rankings = []

# for idx, cod in enumerate(all_cods):
#     url = URL_TEMPLATE.replace('{{ cod }}', str(cod))
#     print(f"Baixando ranking de {url} ({idx + 1} de {num_cods})")
#     r = requests.get(url)
#     if r.status_code != 200:
#         print(f"[ERROR] Erro ao baixar esse ranking. Status code: {r.status_code}")
#         continue

#     soup = BeautifulSoup(r.content, 'html.parser')

#     title = soup.title.text.split('Top 100 Players ')[1].split(' FIDE')[0]
#     year, month = title.split(' ')

#     ratings = []
#     table = soup.find(id='main-col').find_all('table')[1].find('table')
#     for row in table.find_all('tr'):
#         cols = row.find_all('td')
#         player_name = cols[1].text.strip()
#         if player_name == 'Name':
#             continue
#         rating = int(cols[4].text.strip())
#         ratings.append([player_name, rating])

#     rankings.append({
#         'date_human': title,
#         'year': year,
#         'month': int(month),
#         'ratings': ratings,
#     })
    
# with open('./ratings.json', 'w') as fp:
#     json.dump(rankings, fp)

# print('Fim!')

Baixando ranking de https://ratings.fide.com/toparc.phtml?cod=241 (1 de 131)
Baixando ranking de https://ratings.fide.com/toparc.phtml?cod=245 (2 de 131)
Baixando ranking de https://ratings.fide.com/toparc.phtml?cod=249 (3 de 131)
Baixando ranking de https://ratings.fide.com/toparc.phtml?cod=253 (4 de 131)
Baixando ranking de https://ratings.fide.com/toparc.phtml?cod=257 (5 de 131)
Baixando ranking de https://ratings.fide.com/toparc.phtml?cod=261 (6 de 131)
Baixando ranking de https://ratings.fide.com/toparc.phtml?cod=265 (7 de 131)
Baixando ranking de https://ratings.fide.com/toparc.phtml?cod=269 (8 de 131)
Baixando ranking de https://ratings.fide.com/toparc.phtml?cod=273 (9 de 131)
Baixando ranking de https://ratings.fide.com/toparc.phtml?cod=277 (10 de 131)
Baixando ranking de https://ratings.fide.com/toparc.phtml?cod=281 (11 de 131)
Baixando ranking de https://ratings.fide.com/toparc.phtml?cod=285 (12 de 131)
Baixando ranking de https://ratings.fide.com/toparc.phtml?cod=289 (13 de 